In [ ]:
#Functions
import os,ast

os.chdir(r"/home/")
import pandas as pd
import numpy as np

def splitBlogs(inputDataFrame):
    id_list = list(set(inputDataFrame['hardcode_id'].tolist()))
    np.random.seed(1)
    np.random.shuffle(id_list)
    train_id_list = id_list[:int(len(id_list) * 0.8)]
    test_id_list = id_list[int(len(id_list) * 0.8):]
    out_train = inputDataFrame[inputDataFrame['hardcode_id'].isin(train_id_list)]
    out_test = inputDataFrame[inputDataFrame['hardcode_id'].isin(test_id_list)]
    return out_train, out_test

#Generate the feature for each article and unsupervised clustering.
def feature_build(input_DF):
    input_DF = input_DF.copy()
    feature_total = []
    t = 0
    for i in input_DF.index:
        thisLine = [input_DF.loc[i, 'IOC Statistics: IOC Words Number in CTI sentences'],
                    input_DF.loc[i, 'IOC Statistics: IOC Words Number in All sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  Rate of CTI Sentences '],
                    input_DF.loc[i, 'Sentence Statistics:  CTi Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  All Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  tactic Sentences']]
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences']))
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences (Exist or Not)']))
        if t == 0:
            weight_dimension = len(thisLine)
            print(weight_dimension)
            t = 1
        thisLine.extend((input_DF.loc[i, 'Model Result Embedding_PCA']))
        thisLine.extend((input_DF.loc[i, ' CTI Sentence Embedding_PCA']))
        feature_total.append(thisLine)
    for weight_value in [30]:
        for cluster_number in [100]:
            for min_samples_value in [3]:
                import matplotlib.pyplot as plt
                import hdbscan
                from sklearn.preprocessing import StandardScaler
                X = StandardScaler().fit_transform(feature_total)
                X = X.tolist()
                #convert X to list of list
                for i in X:
                    for j in range(len(i)):
                        if j <= weight_dimension:
                            i[j] = i[j] * weight_value
                feature_total = np.array(X)
                clusterer = hdbscan.HDBSCAN(min_cluster_size=cluster_number, min_samples=min_samples_value,
                                            cluster_selection_method='leaf', prediction_data=True).fit(feature_total)
                y_pred = hdbscan.HDBSCAN(min_cluster_size=cluster_number, min_samples=min_samples_value,
                                         cluster_selection_method='leaf').fit_predict(feature_total)
                from sklearn.manifold import TSNE
                projection = TSNE(n_components=2).fit_transform(feature_total)
                from matplotlib.axes._axes import _log as matplotlib_axes_logger
                matplotlib_axes_logger.setLevel('ERROR')
                plt.figure(figsize=(40, 40))
                from matplotlib.cm import get_cmap
                colors = get_cmap("tab20").colors + get_cmap("tab20b").colors
                for i, c in zip(list(set(y_pred)), colors):
                    plt.scatter(projection[y_pred == i, 0], projection[y_pred == i, 1], c=c)
                plt.legend(list(set(y_pred)))
                from collections import Counter
                print(Counter(y_pred))
                input_DF['HDBSCAN cluster result'] = y_pred
    #deepcopy input_DF
    return input_DF, clusterer


def findvalue(in_dict, x):
    if x in in_dict.keys():
        return str(in_dict[x])
    else:
        return 'NaN'


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def probility(x):
    return softmax(x)[1]


def build_feature(input_DF):
    input_DF = input_DF.copy()
    feature_total = []
    t = 0
    for i in input_DF.index:
        thisLine = [input_DF.loc[i, 'IOC Statistics: IOC Words Number in CTI sentences'],
                    input_DF.loc[i, 'IOC Statistics: IOC Words Number in All sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  Rate of CTI Sentences '],
                    input_DF.loc[i, 'Sentence Statistics:  CTI Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  All Sentences'],
                    input_DF.loc[i, 'Sentence Statistics:  tactic Sentences']]
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences']))
        thisLine.extend((input_DF.loc[i, 'Tactics Statistics: CTI Sentences (Exist or Not)']))
        if t == 0:
            weight_dimension = len(thisLine)
            print(weight_dimension)
            t = 1
        thisLine.extend((input_DF.loc[i, 'Model Result Embedding_PCA']))
        thisLine.extend((input_DF.loc[i, ' CTI Sentence Embedding_PCA']))
        feature_total.append(thisLine)
    for weight_value in [30]:
        print('weight_value', weight_value)
        from sklearn.preprocessing import StandardScaler
        X = StandardScaler().fit_transform(feature_total)
        X = X.tolist()
        #convert X to list of list
        for i in X:
            for j in range(len(i)):
                if j <= weight_dimension:
                    i[j] = i[j] * weight_value
        #convert X to array
        feature_total = np.array(X)
        return feature_total

In [ ]:
#Load data
import pickle
import numpy as np
import pandas as pd

data_part1 = pd.read_csv('article_part1.csv',encoding='utf-8-sig',converters={'Sentence Statistics:  Rate of CTI Sentences':ast.literal_eval,'Tactics Statistics: CTI Sentences':ast.literal_eval,'Tactics Statistics: CTI Sentences (Exist or Not)':ast.literal_eval,'Sentence Statistics:  All Sentences':ast.literal_eval,'Sentence Statistics:  tactic Sentences':ast.literal_eval,' Model Result Embedding_PCA':ast.literal_eval,'CTI Sentence Embedding_PCA':ast.literal_eval})
uselessdata,feature_vector=feature_build(data_part1,HDBSCANCluster)
data_part1['embeddingvector']=feature_vector.tolist()

data_part2 = pd.read_csv('article_part2.csv',encoding='utf-8-sig',converters={'Sentence Statistics:  Rate of CTI Sentences':ast.literal_eval,'Tactics Statistics: CTI Sentences':ast.literal_eval,'Tactics Statistics: CTI Sentences (Exist or Not)':ast.literal_eval,'Sentence Statistics:  All Sentences':ast.literal_eval,'Sentence Statistics:  tactic Sentences':ast.literal_eval,' Model Result Embedding_PCA':ast.literal_eval,'CTI Sentence Embedding_PCA':ast.literal_eval})
uselessdata,feature_vector=feature_build(data_part2,HDBSCANCluster)
data_part2['embeddingvector']=feature_vector.tolist()

data_part3 = pd.read_csv('article_part3.csv',encoding='utf-8-sig',converters={'Sentence Statistics:  Rate of CTI Sentences':ast.literal_eval,'Tactics Statistics: CTI Sentences':ast.literal_eval,'Tactics Statistics: CTI Sentences (Exist or Not)':ast.literal_eval,'Sentence Statistics:  All Sentences':ast.literal_eval,'Sentence Statistics:  tactic Sentences':ast.literal_eval,' Model Result Embedding_PCA':ast.literal_eval,'CTI Sentence Embedding_PCA':ast.literal_eval})
uselessdata,feature_vector=feature_build(data_part3,HDBSCANCluster)
data_part3['embeddingvector']=feature_vector.tolist()

data_part3.rename(columns={'hardcode_id': 'Article ID'}, inplace=True)
data_part2.rename(columns={'hardcode_id': 'Article ID'}, inplace=True)
import pandas as pd
import pickle
data_labeled=pd.read_csv('labeleddata.csv',encoding='utf-8-sig')

merged_data = pd.concat([data_part3[['Article ID', 'embeddingvector']], data_part2[['Article ID', 'embeddingvector']]], ignore_index=True)

merged_data.drop_duplicates(subset='Article ID', keep='first', inplace=True)

data_labeled = data_labeled.merge(merged_data, on='Article ID', how='left')

In [ ]:
#Model training and evaluation
df=data_labeled
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

kfold = StratifiedKFold(n_splits=5, shuffle=True)

# initialize lists to store final predictions and true values
final_real= []
final_pred = []

# loop over K-fold splits
for train_idx, test_idx in kfold.split(df['embeddingvector'], df['Label']):
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    model = Sequential()
    model.add(Dense(128, input_dim=250, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(np.array(train_df['embeddingvector'].tolist()), np.array(train_df['Label']), epochs=10, batch_size=32)

    y_pred = model.predict(np.array(test_df['embeddingvector'].tolist())).flatten()
    y_true = np.array(test_df['Label'])

    final_real.extend(y_true)
    final_pred.extend(y_pred)

In [ ]:
#Draw the curve of precision, recall and F1-score in different threshold
import matplotlib.pyplot as plt
import pandas as pd
# Set font size
plt.rcParams.update({'font.size': 30})
results_df = pd.DataFrame(columns=['Threshold', 'Precision', 'Recall', 'F1-Score'])
for threshold in np.arange(0.01, 1.0, 0.01):
    y_pred_binary = np.where(np.array(final_pred) > threshold, True, False)
    precision = precision_score(final_real, y_pred_binary)
    recall = recall_score(final_real, y_pred_binary)
    f1 = f1_score(final_real, y_pred_binary)
    results_df = results_df.append({'Threshold': threshold, 'Precision': precision, 'Recall': recall, 'F1-Score': f1}, ignore_index=True)
plt.figure(figsize=(14, 10))
line_width = 5
plt.plot(results_df['Threshold'], results_df['Precision'], label='Precision', linewidth=line_width)
plt.plot(results_df['Threshold'], results_df['Recall'], label='Recall', linewidth=line_width)
plt.plot(results_df['Threshold'], results_df['F1-Score'], label='F1-Score', linewidth=line_width)
plt.axvline(x=0.15, color='r', linestyle='--', linewidth=2)
plt.grid(True)
plt.legend()
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision, Recall and F1-Score vs. Threshold')
plt.show()